# hw 3

## A randomized non-homogeneous poisson process

Let's consider the idea of non-homogeneous poisson process (this is a slight abuse of the term) with a randomized intensity function $R(t)$. 

Let's look at one such model:

- there is a baseline intensity parameter $\lambda > 0$
- when the $i$-th event occurs, we generate a random variable $M_i \sim \exp(\lambda)$ called the event's "mark" that determines the instaneous increase in the intensity function. Assume the $M_i$ are independent.
- there is a decay parameter, $\alpha > 0$ that determines how long the impact of an event's mark is felt

Suppose there have been $N(t)$ events by time $t$, at times $S_1 < ... < S_{N(t)}$ with marks $M_1,...,M_{N(t)}$

$R(t) = \lambda + \sum_{i=1}^{N(t)} M_i \exp(-\alpha(t - S_i))$, meaning the intensity at time t, is a function of the number of events, their specific times, and their marks.

Using the algorithm suggested in the hints below, write a function generate_process(lam, mu, alpha, T) that generates the arrival times for one run of such a process until time $T$. The output should be a numpy array. 

Output $n=10,000$ runs of such a process until $T=1$ for the parameters $\lambda=2$, $\mu=1$, $\alpha=0.5$. This should be a list (of length n) of numpy arrays (of varying lengths). Calculate N(1) for each run and output that as well. This should be a numpy array of length n.

Hints: 
(1) What is the distribution of the first arrival time?
(2) at $t>0$, if we know the previous arrival times and their marks, how can we use the interarrival method coupled with the thinning method to sample from this process?

In [ ]:
import numpy as np
from scipy.stats import expon, uniform

def generate_process(lam, mu, alpha, T):
    S = np.array([])
    M = np.array([])
    t = 0
    R_max = lam
    while True:
        dt = expon.rvs(scale=1/R_max)
        t += dt
        if t < T:
            R_t = lam + np.sum(M * np.exp(-alpha * (t - S)))
            p_t = R_t / R_max # thining method
            if uniform.rvs() < p_t:
                S = np.append(S, t)
                M_i = expon.rvs(scale=1/mu)
                M = np.append(M, M_i)
                R_max = R_t + M_i 
            else:
                R_max = R_t
        else:
            break
    return S

lam = 2
mu = 1
alpha = 0.5
t = 1

runs = [generate_process(lam, mu, alpha, t) for i in range(0, 10_000)]
N_1 = np.array([len(x) for x in runs])
runs, N_1

## hw4

## Problem

A wrestling tournament has 10 weight classes (we'll design them A, B, C,...,J), each with a sixteen competitor bracket - the tournament is single elimination, with four rounds. For each weight class: 8 matches in the first round, 4 matches in the second round, 2 matches in the third (semi-final) round, and 1 match in the fourth (final) round.

The matches are all given a number: matches 0-7 are the first round in weight class A, 8-15 are the first round matches in weight class B, ..., 72-79 are the first round in weight class J, 80-83 are the second round in weight class A, 84-87 are the second round in weight class B,..., 116-119 are the second round in weight class J, 120-121 are the third round matches for weight class A, etc (there are 150 matches in total).

There are four mats (we'll call them mats 0, 1, 2, 3). For the purposes of giving competitors time to get ready, each mat has a queue (in terms of match numbers) of length 3 (in addition to the competitors on the mat). Once a match is completed, the first match in the queue for that particular mat goes into "service", while the other two matches in the queue for that mat move up, and the next match numerically that is not currently in a queue for a mat will enter the queue for that mat.

The day starts the following way:

Mat 0: Match 0 (in service), Match 4, 8, 12 in queue
Mat 1: Match 1 (in service), Match 5, 9, 13 in queue
Mat 2: Match 2 (in service), Match 6, 10, 14 in queue
Mat 3: Match 3 (in service), Match 7, 11, 15 in queue

If Match 3 finishes before the other matches, then match 16 will enter the queue for Mat 3, if 2 finishes next, then match 17 will enter the queue for Mat 2, etc. At the end of the day, when there are not enough matches to fill up all the queue, the next match will enter the queue for the mat with the shortest queue length.

The length of time each match (in minutes) follows a gamma distribution with shape parameter $3$ and rate parameter $0.35$.

Run 1000 simulations of the queueing system, and calculate the amount of time until the tournament is complete. Suppose each mat requires 1 referee who is paid USD 60/hour, and a timekeeper, and a scorekeeper who are each paid USD 15/hour. If each entrant to the tournament pays a USD 25 entrance fee (there 160 competitors), how much money does the tournament organizer expect to make or lose? What are the 5th and 95th percentiles of this quantity?

(Note: don't worry about pathological outcomes like say, the final for a division going before one of the semi-finals is complete) 

In [ ]:
import numpy as np
import simpy

from scipy.stats import gamma, expon

def match(shared):
    main_request = shared['main'].request()
    yield main_request
    q_len = [len(shared['mats'][i].queue) + shared['mats'][i].count for i in range(4)]
    ind = np.argmin(q_len)
    rqt = shared['mats'][ind].request()
    yield rqt
    st = gamma.rvs(3, scale=1/0.35)
    yield shared['env'].timeout(st)
    shared['mats'][ind].release(rqt)
    shared['main'].release(main_request)
    shared['end_times'].append(shared['env'].now)


def simulate():
    env = simpy.Environment()
    main = simpy.Resource(env=env, capacity=16)
    mats = [simpy.Resource(env=env, capacity=1) for i in range(4)]
    end_times = []
    shared = {'env': env,
              'main': main,
              'mats': mats,
              'end_times': end_times}
    for i in range(0, 150):
        env.process(match(shared))
    env.run()
    return max(shared['end_times'])


def run_sims(nsims):
    X = np.array([simulate() for _ in range(nsims)])
    cost = 4 * X * (60 + 15 + 15) / 60
    revenue = 25 * 160
    profit = revenue - cost
    return profit

### output numpy array of simulations here

profits = run_sims(1_000)

### output 5th percentile and 95th percentile

p5 = np.percentile(profits, 5)
p95 = np.percentile(profits,95)


## hw 5

## Problem

Consider a take-out restaurant. It has a capacity of 30 people combined that can be in line, getting served, waiting for food after placing their order, or working in the restaurant.

Customers arrive according to a poisson process with intensity $\lambda_\text{arrival}$; if the store is at capacity, they leave.

There is a single line for the checkout counter(s). The amount of time it takes to ring someone up is exponential distribution with rate $\lambda_\text{checkout}$. The net revenue in USD of an order follows a gamma distribution with shape parameter $2$ and rate $\lambda_\text{rev}$.

There is also a single queue for the kitchen. There are two kinds of cooks, master cooks, and line cooks. An order goes to whomever is available first; with the order going to a line cook if no one is working. The amount of time it takes each to make an order is exponential with rates $\lambda_\text{master}$ and $\lambda_\text{line}$ respectively.

A checkout worker costs USD 15 per hour, a line cook costs USD 25 / hour, and a master cook costs 90 / hour.

Run the simulation, serving all customers that that arrive before one hour is (do not worry about paying overtime, to make this easy, assume they only get paid for one hour regardless of when they finish). Find the optimal combination of workers to maximize profits (total net revenue minus costs of workers). Use 50 function evaluations with hyperopt to do this. Each evaluations of the objective function should do 50 simulations of the system.

Use the following variance reduction techniques simultaneously:

- common random numbers
- conditioning on the number of people served
- two control variables (mean interarrival time, mean checkout time)

Fill out the following code skeleton, and do not change the names of the function or the arguments.

In [ ]:
import hyperopt
import numpy as np
import simpy

T = 1
lam_arrival = 120
lam_checkout = 100
lam_line = 40
lam_master = 90
lam_cook = [lam_line, lam_master]
lam_rev = 0.5

costs = np.array([15, 25, 90])

def simulate(n):
    def arrivals():
        while True:
            U_st = uniform.rvs(size=2)
            dt = expon.rvs(scale=1/lam_arrival)
            cvs['dt'].append(dt)
            rev = 2 / lam_rev
            if env.now + dt < T:
                yield env.timeout(dt)
                if store.count < n_capacity:
                    store_rqt = store.request()
                    env.process(service(U_st, store_rqt))
                    revenue.append(rev)
            else:
                break
            
    def service(U, store_rqt):
        rqt_checkout = checkout.request()
        yield rqt_checkout
        st_checkout = expon.ppf(U[0], scale=1/lam_checkout)
        cvs['st'].append(st_checkout)
        yield env.timeout(st_checkout)
        checkout.release(rqt_checkout)
        rqt_cook0 = cook0.request()
        rqt_cook1 = cook1.request()
        results = yield simpy.AnyOf(env, [rqt_cook0, rqt_cook1])
        if rqt_cook0 in results:
            if rqt_cook1 in results:
                cook1.release(rqt_cook1)
            else:
                rqt_cook1.cancel()
            st_cook = expon.ppf(U[1], scale=1/lam_cook[0])
            yield env.timeout(st_cook)
            cook0.release(rqt_cook0)
        else:
            rqt_cook0.cancel()
            st_cook = expon.ppf(U[1], scale=1/lam_cook[1])
            yield env.timeout(st_cook)
            cook1.release(rqt_cook1)
        store.release(store_rqt)

    n_workers = np.sum(n)
    env = simpy.Environment()
    n_capacity = 30 - n_workers
    store = simpy.Resource(env=env, capacity=n_capacity)
    checkout = simpy.Resource(env=env, capacity=n[0])
    cook0 = simpy.Resource(env=env, capacity=n[1])
    cook1 = simpy.Resource(env=env, capacity=n[2])
    revenue = []
    cvs = {'dt': [], 'st': []}
    env.process(arrivals())
    env.run()
    total_revenue = np.sum(revenue) - np.sum(T * costs * n)
    dt = np.mean(cvs['dt']) - 1 / lam_arrival
    st = np.mean(cvs['st']) - 1 / lam_checkout
    return total_revenue, dt, st

def run_simulations(n):
    rev = np.zeros(50)
    dt = np.zeros(50)
    st = np.zeros(50)
    np.random.seed(42)
    for i in range(50):
        rev[i], dt[i], st[i] = simulate(n)
    cd = - np.cov(rev, dt)[0][1] / np.var(dt)
    cs = - np.cov(rev, st)[0][1] / np.var(st)
    rev_mod = rev + cd * dt + cs * st
    return rev_mod

def obj(n):
    rev = run_simulations(n)
    return (-1) * np.mean(rev)

best_setup = hyperopt.fmin(fn=obj,
                           space=[hyperopt.hp.quniform('n{}'.format(i), 1, 10, 1) for i in range(3)],
                           algo=hyperopt.tpe.suggest,
                           max_evals=50)

best_setup